In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.model_selection import GridSearchCV
np.random.seed(0)
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import sklearn

import category_encoders as ce
from catboost import CatBoostClassifier

In [3]:

def scaler(scaler, data, test=None):
    scaler.fit(data)  # Apply transform to both the training set and the test set.
    train_scale = scaler.transform(data)
    if test is not None:
        test_scale = scaler.fit_transform(test)

    return train_scale, test_scale, scaler

def train_model(classifier, X_tr, y_tr, X_te, y_te):
    print('start training...')
    classifier.fit(X_tr, y_tr)
    print('evaluation...')
    y_p = classifier.predict(X_te)
    score = evaluate(y_te, y_p)
    print(f'score is {score}')
    return classifier, score

def evaluate(y_true, y_pred):
    return f1_score(y_true, y_pred, average="macro")

In [4]:
data = pd.read_table('data/dev.tsv')
test = pd.read_table('data/eval.tsv')

In [5]:
df = data.copy()

In [6]:
eval = test.copy()

In [7]:
from scipy import stats
def happy_sad(x):
    if x>df['valence'].mean():
        return 'happy'
    else:
        return 'sad'

def popularity_cat(x):
    if x>= 7:
        return 'high'
    elif x >= 4 and x < 7:
        return 'med'
    else:
        return 'low'

df['boringness'] = df['loudness'] + df['tempo'] + (df['energy']*100) + (df['danceability']*100)
df['valence_happy_sad'] = df['valence'].apply(lambda x: happy_sad(x))
df['loudness_plus_60'] = df['loudness'].apply(lambda x: x+60)
df['loudness_pos'] = df['loudness'].apply(lambda x: -1*x)
df['loudness_pos'] = np.sqrt(df['loudness_pos'])
df['boringness_plus_60'] = df['boringness'].apply(lambda x: x+60)
df['duration_ms_box_cox_trans'] = stats.boxcox(df['duration_ms'])[0]
df['acousticness_sqrt_trans'] = np.sqrt(df['acousticness'])
df['liveness_sqrt_trans'] = np.sqrt(df['liveness'])
df['popularity_sqrt_trans'] = np.sqrt(df['popularity'])
df['popularity_sqrt_trans_cat'] = df['popularity_sqrt_trans'].apply(lambda x: popularity_cat(x))
df['speechiness_sqrt_trans'] = np.sqrt(df['speechiness'])

/home/omid/anaconda3/envs/ml/lib/python3.8/site-packages/pandas/core/series.py:726: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [8]:
# df = df.sort_values(by='mode')
# mode0 = stats.boxcox(df[df['mode']==0]['duration_ms'])[0]
# mode1 = stats.boxcox(df[df['mode']==1]['duration_ms'])[0]
#

In [9]:
# df.loc[df['mode']==0,['duration_ms_box_cox_trans_per_class']] = mode0
# df.loc[df['mode']==1,['duration_ms_box_cox_trans_per_class']] = mode1

In [10]:
df = df.fillna(value=0)
# df.describe().T

In [11]:
import seaborn as sns
import matplotlib.pyplot as plt
def dist_plot_box_cox_by_class(df,col):
    plt.figure(figsize=(16,6))
    plt.title("Distribution of "+col+" box cox transformation")
    sns.distplot(df[df['mode']==0][col],
                 color="green", kde=True,bins=120, label='mode 0')
    sns.distplot(df[df['mode']==1][col],color="red", kde=True,bins=120, label='mode 1')
    plt.legend()
    plt.show()
def dist_plot_box_cox(df,col):
    plt.figure(figsize=(16,6))
    plt.title("Distribution of "+col+" box cox transformation")
    sns.distplot(stats.boxcox(df[col])[0],
                 color="green", kde=True,bins=120, label='mode 0')
    plt.legend()
    plt.show()
# dist_plot_box_cox_by_class(df,'duration_ms_box_cox_trans_per_class')

In [12]:

col = [
    'valence',
        'year',
        # 'acousticness',
        'artists',
         'danceability',
       'duration_ms',
        'energy',
        'explicit',
        # 'id',
        'instrumentalness',
        'key',
       'liveness',
        # 'loudness',
        # 'popularity',
        # 'speechiness',
        'tempo',
       #  'mode',
       # 'loudness_plus_60',
        'loudness_pos',
         # 'boringness',
       #  'valence_happy_sad',
       # 'boringness_plus_60',
       #  'duration_ms_box_cox_trans',
       'acousticness_sqrt_trans',
       #  'liveness_sqrt_trans',
       # 'popularity_sqrt_trans',
        'speechiness_sqrt_trans',
          # 'duration_ms_box_cox_trans_per_class',
    # 'popularity_sqrt_trans_cat'
        ]

In [13]:
df = sklearn.utils.shuffle(df)

In [14]:
# print(f"{len(df.loc[(df['duration_ms_box_cox_trans']<12.5) | (df['duration_ms_box_cox_trans']>15)])}")
# duration_df = df.loc[(df['duration_ms_box_cox_trans'] > 12.5) & (df['duration_ms_box_cox_trans']<15)]
# sns.distplot(duration_df['duration_ms_box_cox_trans'],
#                  color="green", kde=True,bins=120, label='mode 0')

In [15]:
# sns.countplot(duration_df['mode'], palette='Set3')

In [16]:
# sns.countplot(df.loc[(df['duration_ms_box_cox_trans']<12.5) | (df['duration_ms_box_cox_trans']>15)]['mode'], palette='Set3')

In [17]:
X = df[col]
y = df['mode']


# encoder = ce.OneHotEncoder(cols=['artists'])
# X = encoder.fit_transform(X,y)

In [18]:
X

,valence,year,artists,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,tempo,loudness_pos,acousticness_sqrt_trans,speechiness_sqrt_trans
29176,0.708,1936,['Robert Johnson'],0.805,135467,0.0685,0,0.000337,3,0.1100,115.303,3.712008,0.981326,0.352136
106871,0.726,1964,['Lefty Frizzell'],0.640,178280,0.3830,0,0.000307,4,0.1790,122.054,3.623534,0.813019,0.189737
41122,0.126,1983,"['Richard Wagner', 'Wiener Philharmoniker', 'S...",0.235,183667,0.3650,0,0.907000,7,0.1700,91.187,3.723305,0.962808,0.188680
101401,0.889,1972,['Albert Hammond'],0.626,186347,0.6710,0,0.000000,5,0.0718,95.546,3.090469,0.503984,0.170000
56863,0.162,2011,"['Mandy Moore', 'Zachary Levi']",0.231,222120,0.1300,0,0.000001,5,0.0832,87.418,3.831579,0.921954,0.176068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41993,0.240,1994,['Pink Floyd'],0.496,370667,0.7710,0,0.009290,4,0.0788,100.298,3.455141,0.433590,0.229783
97639,0.140,1986,['Queen'],0.240,272667,0.8590,0,0.220000,1,0.3450,83.804,3.373426,0.322490,0.357771
95939,0.887,1997,['Grupo Exterminador'],0.797,229427,0.7790,0,0.000000,7,0.1130,116.832,2.609214,0.819756,0.403733
117952,0.557,2009,['Anika Noni Rose'],0.535,144827,0.4480,0,0.000013,0,0.3030,78.784,2.785498,0.898888,0.207605


In [19]:
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler,NearMiss,TomekLinks,ClusterCentroids
from imblearn.over_sampling import SMOTE,SVMSMOTE,ADASYN
from imblearn.combine import SMOTETomek

# ous = RandomUnderSampler(random_state=42)
# nm = NearMiss()
# tl = TomekLinks()
# cc = ClusterCentroids(random_state=42,n_jobs=-1)/
# smt = SMOTETomek(sampling_strategy='auto',n_jobs=-1)
# over = SMOTE(sampling_strategy=0.1,random_state=42)
# under = RandomUnderSampler(sampling_strategy=0.5,random_state=42)
#
#
# steps = [('o', over), ('u', under)]
# pipeline = Pipeline(steps=steps)
# sm = SVMSMOTE()
# X_spl, y_spl = nm.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=0,shuffle=True)

#
# encoder = ce.(cols=['key'])
# encoder.fit(X_train,y_train)
# X_train = encoder.transform(X_train, y_train)
# X_test = encoder.transform(X_test, y_test)

In [20]:
# from sklearn.preprocessing import *
# X_train_scal, X_test_scal, x_scaler = scaler(StandardScaler(), X_train, X_test)

In [21]:
from sklearn.decomposition import PCA
def dim_reduction(train, test=None):
    tsvd = PCA(n_components=4,random_state=42)
    tsvd.fit(train)
    print('strat transformation SVD')
    X_train_svd = tsvd.transform(train)
    X_test_svd = None
    if test is not None:
        X_test_svd = tsvd.transform(test)
    percentage = np.sum(tsvd.explained_variance_ratio_) * 100
    print(f'{percentage:.2f}%')


    return X_train_svd, X_test_svd,tsvd

# X_train_svd, X_test_svd , tsvd = dim_reduction(X_train_scal, X_test_scal)


In [35]:
from collections import Counter

counter = Counter(y)

# estimate scale_pos_weight value
estimate = counter[0]/counter[1]
print('Estimate: %.3f' % estimate)
print(counter,counter[0])

Estimate: 0.415
Counter({1: 96508, 0: 40014}) 40014


In [48]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
# clf = SVC(random_state=42)
# clf = GaussianNB()
# clf = MultinomialNB(alpha=10/100)
# clf = BernoulliNB(alpha=10/100)
# clf = LogisticRegression(penalty='l2',max_iter=10000,random_state=0)
# clf = RandomForestClassifier(criterion = 'entropy',n_estimators=1500,max_depth=1,n_jobs=-1,random_state=0)
# clf = GradientBoostingClassifier(n_estimators=1000, random_state=0)
# clf = AdaBoostClassifier(n_estimators=100,random_state=0)
# clf = SGDClassifier()
# clf = KNeighborsClassifier(n_neighbors = 10)
# clf = MLPClassifier(early_stopping=True,random_state=1,max_iter=500, verbose=True)
# clf = LGBMClassifier(learning_rate=0.00001,n_jobs=-1,n_estimators=1000)
# clf= XGBClassifier(learning_rate=0.65, scale_pos_weight=estimate,n_jobs=-1,random_state=0,)
clf = CatBoostClassifier(auto_class_weights='SqrtBalanced',random_state=0,)
# clf = VotingClassifier(estimators=[('RandomForestClassifier', RandomForestClassifier(n_estimators=1000,n_jobs=-1, random_state=0)),
#                                    ('XGBClassifier', XGBClassifier(learning_rate=0.1,n_jobs=-1,random_state=0)),
#                                    ('LGBMClassifier', LGBMClassifier(learning_rate=0.15,n_jobs=-1,n_estimators=1000))],
#                        voting='soft', weights=[1, 2, 1])

print(f'start training... {clf}')
# X_train_val, X_val, y_train_val, y_val = train_test_split(X_train_scal, y_train,
#                                                     test_size=0.1,
#                                                   random_state=0,shuffle=True)
# eval_set = [(X_val, y_val)]
# clf.fit(X_train_val, y_train_val,early_stopping_rounds=50,eval_metric="mae", eval_set=eval_set, verbose=True)
clf.fit(X_train,y_train,cat_features=['artists','key'])
print('evaluation...')
y_p = clf.predict(X_test)
score = evaluate(y_test, y_p)
print(f'score is {score}')
# scores = cross_val_score(clf, X, y, cv=5,scoring='f1_macro')
# print(f'mean score {np.mean(scores)}, max score {np.max(scores)}, min score {np.min(scores)}')

start training... <catboost.core.CatBoostClassifier object at 0x7f8e54625a60>
Learning rate set to 0.076434
0:	learn: 0.6831313	total: 35.7ms	remaining: 35.7s
1:	learn: 0.6745071	total: 57.8ms	remaining: 28.8s
2:	learn: 0.6677643	total: 77.3ms	remaining: 25.7s
3:	learn: 0.6615662	total: 111ms	remaining: 27.6s
4:	learn: 0.6559369	total: 142ms	remaining: 28.3s
5:	learn: 0.6510332	total: 171ms	remaining: 28.4s
6:	learn: 0.6448152	total: 190ms	remaining: 27s
7:	learn: 0.6400497	total: 203ms	remaining: 25.2s
8:	learn: 0.6354553	total: 226ms	remaining: 24.8s
9:	learn: 0.6315255	total: 237ms	remaining: 23.4s
10:	learn: 0.6278331	total: 251ms	remaining: 22.6s
11:	learn: 0.6251555	total: 262ms	remaining: 21.6s
12:	learn: 0.6225865	total: 279ms	remaining: 21.2s
13:	learn: 0.6205060	total: 299ms	remaining: 21.1s
14:	learn: 0.6167042	total: 321ms	remaining: 21.1s
15:	learn: 0.6131110	total: 341ms	remaining: 21s
16:	learn: 0.6102116	total: 355ms	remaining: 20.5s
17:	learn: 0.6075488	total: 369ms	re

In [49]:
print(f' score {score} {clf}')

 score 0.684066543939522 <catboost.core.CatBoostClassifier object at 0x7f8e54625a60>


In [25]:
# from sklearn.neighbors import KNeighborsClassifier
#
# score = []
# for i in range(2,100,2):
#     knn = KNeighborsClassifier(n_neighbors = i)
#     knn.fit(X_train, y_train)
#     y_pred = knn.predict(X_test)
#     sco = evaluate(y_test, y_pred)
#     score.append(sco)
#     print(f'{i} {sco:.4f}')

In [26]:
eval

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,popularity,speechiness,tempo
0,0.741,1962,0.9950,['Frank Proffitt'],0.465,147173,0.389,0,0lQ4HSTCEsNrEzqMJM3bKV,0.506000,11,0.1100,-14.770,10,0.0395,125.010
1,0.928,1938,0.9180,"['Billie Holiday', 'Teddy Wilson']",0.578,165133,0.234,0,4VTfF39cEPGEP7PIyQA2eS,0.000391,0,0.0784,-12.305,6,0.1020,172.403
2,0.733,1998,0.1270,['Monica'],0.681,333960,0.726,0,3GkclSNFTsqpTZJhzK9FCy,0.000019,9,0.1190,-8.421,37,0.0358,147.925
3,0.914,2001,0.1530,['Los Acosta'],0.724,224427,0.731,0,0yjSicm6ScdAYF8yUJpVaq,0.003430,8,0.1770,-7.159,43,0.0325,101.602
4,0.228,2015,0.0291,['Lil Uzi Vert'],0.594,172704,0.850,1,7pudclCl5P99Me3P5F9hPC,0.000000,2,0.1100,-6.525,60,0.0954,100.002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34126,0.783,1965,0.5700,['Bob Dylan'],0.770,155427,0.646,0,1kvqv4axiWkxOdRz017sQZ,0.000000,9,0.1370,-10.723,37,0.0684,118.771
34127,0.425,1975,0.2530,['Camel'],0.373,320387,0.423,0,07wbIsFWkJSfkbVFSsp7Ne,0.527000,0,0.1410,-12.577,36,0.0358,120.941
34128,0.142,2000,0.9590,['Lara Fabian'],0.156,315267,0.279,0,6embXV2TNlwiP0lrVsiwLa,0.007810,0,0.1170,-10.108,51,0.0400,72.555
34129,0.844,1982,0.0214,['Huey Lewis & The News'],0.438,225000,0.418,0,00uWuJtkT7qNBKkhZtgWzF,0.000000,2,0.3300,-12.556,24,0.0394,98.173


In [27]:
def WriteOnFile(name, y_eval):
    f = open(name, "w")
    f.write("Id,Predicted\n")
    for index, i in enumerate(y_eval):
        f.write(f"{index},{i}\n")
    f.close

eval['boringness'] = eval['loudness'] + eval['tempo'] + (eval['energy']*100) + (eval['danceability']*100)
eval['valence_happy_sad'] = eval['valence'].apply(lambda x: happy_sad(x))
eval['loudness_plus_60'] = eval['loudness'].apply(lambda x: x+60)
eval['loudness_pos'] = eval['loudness'].apply(lambda x: -1*x)
eval['loudness_pos'] = np.sqrt(eval['loudness_pos'])
eval['boringness_plus_60'] = eval['boringness'].apply(lambda x: x+60)
eval['duration_ms_box_cox_trans'] = stats.boxcox(eval['duration_ms'])[0]
eval['acousticness_sqrt_trans'] = np.sqrt(eval['acousticness'])
eval['liveness_sqrt_trans'] = np.sqrt(eval['liveness'])
eval['popularity_sqrt_trans'] = np.sqrt(eval['popularity'])
eval['speechiness_sqrt_trans'] = np.sqrt(eval['speechiness'])


eval = eval.fillna(value=0)

/home/omid/anaconda3/envs/ml/lib/python3.8/site-packages/pandas/core/series.py:726: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [51]:
test = eval[col]
# test = encoder.transform(test)
# test_scal = x_scaler.transform(test)
# test_svd = tsvd.transform(test_scal)
y_pred = clf.predict(test)
WriteOnFile('submission.csv',y_pred)

In [29]:
test


,valence,year,artists,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,tempo,loudness_pos,acousticness_sqrt_trans,speechiness_sqrt_trans
0,0.741,1962,['Frank Proffitt'],0.465,147173,0.389,0,0.506000,11,0.1100,125.010,3.843176,0.997497,0.198746
1,0.928,1938,"['Billie Holiday', 'Teddy Wilson']",0.578,165133,0.234,0,0.000391,0,0.0784,172.403,3.507848,0.958123,0.319374
2,0.733,1998,['Monica'],0.681,333960,0.726,0,0.000019,9,0.1190,147.925,2.901896,0.356371,0.189209
3,0.914,2001,['Los Acosta'],0.724,224427,0.731,0,0.003430,8,0.1770,101.602,2.675631,0.391152,0.180278
4,0.228,2015,['Lil Uzi Vert'],0.594,172704,0.850,1,0.000000,2,0.1100,100.002,2.554408,0.170587,0.308869
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34126,0.783,1965,['Bob Dylan'],0.770,155427,0.646,0,0.000000,9,0.1370,118.771,3.274599,0.754983,0.261534
34127,0.425,1975,['Camel'],0.373,320387,0.423,0,0.527000,0,0.1410,120.941,3.546407,0.502991,0.189209
34128,0.142,2000,['Lara Fabian'],0.156,315267,0.279,0,0.007810,0,0.1170,72.555,3.179308,0.979285,0.200000
34129,0.844,1982,['Huey Lewis & The News'],0.438,225000,0.418,0,0.000000,2,0.3300,98.173,3.543445,0.146287,0.198494


In [30]:
# cols = [
#     'valence',
#         'year',
#         # 'acousticness',
#         # 'artists',
#          'danceability',
#        # 'duration_ms',
#         'energy',
#         'explicit',
#         # 'id',
#         'instrumentalness',
#         'key',
#        'liveness',
#         # 'loudness',
#         # 'popularity',
#         # 'speechiness',
#         'tempo',
#        #  'mode',
#        # 'loudness_plus_60',
#         'loudness_pos',
#          # 'boringness',
#        #  'valence_happy_sad',
#        # 'boringness_plus_60',
#         'duration_ms_box_cox_trans',
#        'acousticness_sqrt_trans',
#        #  'liveness_sqrt_trans',
#        # 'popularity_sqrt_trans',
#         'speechiness_sqrt_trans',
#       # 'duration_ms_box_cox_trans_per_class'
#         ]
